In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests 
import pickle

# Get list of all URLs to parse

In [2]:
headers = {
    'authority': 'cdn.cookielaw.org',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.medhelp.org/',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}

In [3]:
def extract_urls_from_medhelp(URL, headers):
    
    response = requests.get(URL, headers=headers)
    if response.status_code != 200:
        return -1
    
    soup = BeautifulSoup(response.content, "html.parser")

    div_rel = soup.find('div', id='body_container')

    all_h2_tags = div_rel.find_all('h2', class_ = 'subj_title')

    post_list = []

    for child in all_h2_tags:
        post_list.append( child.a.attrs['href'])
    
    return post_list

In [4]:
base_url = "https://www.medhelp.org/forums/Eye-Care/show/90?page="

In [5]:
all_pages_post_urls = {}
for i in range(1, 800):
    created_url = base_url + str(i)
    page = i
    post_list = extract_urls_from_medhelp(created_url, headers=headers)
    all_pages_post_urls[page] = post_list
    if page%50 == 0:
        print(f"===== Completed {page} number of pulls =========")



===== Completed 50 number of pulls =========
===== Completed 100 number of pulls =========
===== Completed 150 number of pulls =========
===== Completed 200 number of pulls =========
===== Completed 250 number of pulls =========
===== Completed 300 number of pulls =========
===== Completed 350 number of pulls =========
===== Completed 400 number of pulls =========
===== Completed 450 number of pulls =========
===== Completed 500 number of pulls =========
===== Completed 550 number of pulls =========
===== Completed 600 number of pulls =========
===== Completed 650 number of pulls =========
===== Completed 700 number of pulls =========
===== Completed 750 number of pulls =========


In [7]:
with open('all_pages_post_urls.pickle', 'wb') as f:
    pickle.dump(all_pages_post_urls, f, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
for value in all_pages_post_urls.values():
    if value == -1:
        print("At least one request was returned without 200 status code")

In [17]:
all_urls_for_posts = []
for key, value in all_pages_post_urls.items():
    for url in value:
        all_urls_for_posts.append(url)

In [18]:
with open('all_urls_for_posts.pickle', 'wb') as f:
    pickle.dump(all_urls_for_posts, f, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
len(all_urls_for_posts)

4748

In [24]:
all_urls_for_posts[1]

'/posts/Eye-Care/eyeball-pain/show/1224201'

# Get content for each post, includes question and clinical response

In [25]:
def extract_details_medhelp_post(post_url, headers):
    website = 'https://www.medhelp.org'
    URL = website + post_url + '?sort_by=oldest'
    resp_post = requests.get(URL , headers=headers)
    if resp_post.status_code != 200:
        return -1
    soup = BeautifulSoup(resp_post.content, "html.parser")
    div_rel = soup.find('div', id='body_container')

    post_heading = div_rel.find('h1', class_= 'subj_title').text.strip()
    post_details = div_rel.find('div', id='subject_msg').text.strip()

    reply_details = div_rel.find('div', class_='resp_body').text.strip()

    reply_name = div_rel.find('div', class_= 'resp_info').a.text
    reply_page = div_rel.find('div', class_= 'resp_info').a.attrs['href']

    return [post_heading, post_details, reply_details, reply_name, reply_page]

In [28]:
all_post_details_dict = {}
count = 0
for post_url in all_urls_for_posts:
    return_parse_details = extract_details_medhelp_post(post_url=post_url, headers=headers)
    all_post_details_dict[post_url] = return_parse_details
    if count % 500 == 0:
        print(f"==== Completed extraction of total {count+1} number of posts ====")
    count += 1

==== Completed extraction of total 1 number of posts ====
==== Completed extraction of total 501 number of posts ====
==== Completed extraction of total 1001 number of posts ====
==== Completed extraction of total 1501 number of posts ====
==== Completed extraction of total 2001 number of posts ====
==== Completed extraction of total 2501 number of posts ====
==== Completed extraction of total 3001 number of posts ====
==== Completed extraction of total 3501 number of posts ====
==== Completed extraction of total 4001 number of posts ====
==== Completed extraction of total 4501 number of posts ====


In [29]:
with open('all_post_details_dict.pickle', 'wb') as f:
    pickle.dump(all_post_details_dict, f, protocol=pickle.HIGHEST_PROTOCOL)